In [1]:
# Import libraries
import math
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
from collections import defaultdict
from itertools import chain, combinations
import random

In [ ]:
#clean price field for all records ( by keeping aside the 7500: rows)

az_price22=pd.read_csv('Amazon_Price.csv')
az_price22=pd.DataFrame(az_price22) 
az_price=az_price22.copy()
del az_price['Unnamed: 0']
del az_price['First_Bought_Product']

Price1=az_price.iloc[:7500]
Price2=az_price.iloc[7500:]
Price2.tail()

def cleanprice(flat_price):
    for i in range(0,len(flat_price)):
        for j in range(0,len(flat_price.iloc[i])):
            if (str(flat_price.iloc[i][j]).find('£')>=0):
                start=(str(flat_price.iloc[i][j]).find('£'))+1
                end=len(str(flat_price.iloc[i][j]))
                flat_price.iloc[i][j]=str(flat_price.iloc[i][j][start:end])
            else:
                flat_price.iloc[i][j]=''
    return flat_price

Price3=cleanprice(Price1)

Price4=pd.concat([Price3,Price2], axis=0)
#Price4.head()

#Generate new file for price data
Price4.to_csv('Amazon_Association_Total Data_Price.csv')



In [5]:
price_df = pd.read_csv('Amazon_Association_Total Data_Price.csv')
rating_df = pd.read_csv('Amazon_Average_Rating.csv')
names_df = pd.read_csv('Amazon_Product_Names.csv')

#Delete non required columns
del names_df['Unnamed: 0']
del rating_df['Unnamed: 0']
del price_df['Unnamed: 0']
del names_df['6']
del rating_df['First_Bought_Product']

#Create Names vector or separate data frame with single vector
agg_df = pd.DataFrame(names_df.values.ravel(), columns=['Names'])

#Remove duplicates and Nans
old=list(agg_df.iloc[:,0])

new_prd=set(old)
#print(len(new_prd))
new_prd=list(new_prd)
#print(len(new_prd))
names_noNA=list()

for i in range(0,len(new_prd)):
    if (pd.isna(new_prd[i])):
        continue
    else:
        names_noNA.append(new_prd[i])

len(new_prd), len(names_noNA)


#Get indexes of data from data frame
ind_n=list()
type(ind_n)
for i in range(0,len(names_noNA)):
    num_i=old.index(names_noNA[i])
    ind_n.append(num_i)

#print(ind_n)

#Final names vector for clustering
Final_names_Clustering=agg_df.iloc[ind_n,:]
Final_names_Clustering.head()
len(Final_names_Clustering)

#Price vector for clustering
agg_price_df = pd.DataFrame(price_df.values.ravel(), columns=['Price'])

Final_price_Clustering=agg_price_df.iloc[ind_n,:]
Final_price_Clustering.head()

#Price rating for clustering
agg_rating_df = pd.DataFrame(rating_df.values.ravel(), columns=['Rating'])

Final_rating_Clustering=agg_rating_df.iloc[ind_n,:]
Final_rating_Clustering.head()

#Create new data frame by merging these three frames

final_clustering_df=pd.concat([Final_names_Clustering,Final_price_Clustering,Final_rating_Clustering], axis=1)

final_clustering_df.head()

#Handle values in rating column
def cleanrating_clustering(flat_price):
    for i in range(0,len(flat_price)):
        if(pd.isna(flat_price.iloc[i][2])):
            flat_price.iloc[i,2]=0
        else:
            if(str(flat_price.iloc[i][2]).find('o')==0):
                flat_price.iloc[i,2]=0
            elif(str(flat_price.iloc[i][2]).find('.')<0):
                flat_price.iloc[i,2]=str(flat_price.iloc[i][2])[0:1]
            else:
                flat_price.iloc[i,2]=flat_price.iloc[i][2]
    return flat_price

#Handle values in price column
def cleanprice_clustering(flat_price):
    for i in range(0,len(flat_price)):
        if (str(flat_price.iloc[i][1]).find('<')>=0):
            start=str(flat_price.iloc[i][1]).find('<')
            flat_price.iloc[i,1]=str(flat_price.iloc[i][1][0:start])
        elif(str(flat_price.iloc[i][1]).find('£')>=0):
            y=(str(flat_price.iloc[i][1]).find('£'))-3
            flat_price.iloc[i,1]=str(flat_price.iloc[i][1][0:y])
        elif(str(flat_price.iloc[i][1]).find(',')>=0):
            x=(str(flat_price.iloc[i][1]).find(','))+1
            yy=len(str(flat_price.iloc[i][1]))
            flat_price.iloc[i,1]=str(flat_price.iloc[i][1][x:yy])
        else:
            flat_price.iloc[i,1]=flat_price.iloc[i][1]
    return flat_price

#Execute function to handle price values and convert price to float value

final_clustering_df=cleanprice_clustering(final_clustering_df)
final_clustering_df=cleanprice_clustering(final_clustering_df)
final_clustering_df['Price'] = final_clustering_df['Price'].astype(float)

#Get float values for rating by cleaning it first
final_clustering_df=cleanrating_clustering(final_clustering_df)
final_clustering_df=cleanrating_clustering(final_clustering_df)
final_clustering_df['Rating'] = final_clustering_df['Rating'].astype(float)

#Mean value imputation
mean_value=final_clustering_df['Price'].mean()
final_clustering_df['Price']=final_clustering_df['Price'].fillna(mean_value)
final_clustering_df.isna().sum()
final_clustering_df.head()

,Names,Price,Rating
12305,Fridgemagic A3 Dry Wipe Magnetic Wipe Clean Board,5.750000,4.5
19040,6 Strings Of Tropical Fish,21.741747,0.0
19324,Bristol Novelty Satin Shirt &amp; Ruffles. (ad...,21.741747,0.0
25335,Revell 05404 H.M.S. Bounty Model Kit,19.090000,3.5
1498,Ty UK 6-inch Wishful Beanie Boo Plush,13.990000,4.5


In [6]:
final_clustering_df.to_csv('Amazon_Association_final_clustering_df.csv')

In [52]:
######## Testing data on built in K-Modes algorithm ########### 
######## K-mode clustering with built-in package ##############
from kmodes.kmodes import KModes
km = KModes(n_clusters=5, init='Huang', n_init=11, verbose=1)
clusters = km.fit_predict(final_clustering_df)
labels = km.predict(final_clustering_df)
kmodes = km.cluster_centroids_
shape = kmodes.shape
clusters, kmodes, shape
print(kmodes)

#Print Clusters
cluster_map = pd.DataFrame()
cluster_map['data_index'] = final_clustering_df.index.values
cluster_map['cluster'] = km.labels_

#Print clusters
cluster_map[cluster_map.cluster == 0]

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 0, cost: 42368.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 710, cost: 43372.0
Run 2, iteration: 2/100, moves: 1007, cost: 43372.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 274, cost: 42403.0
Run 3, iteration: 2/100, moves: 0, cost: 42403.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 0, cost: 44615.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 1061, cost: 43366.0
Run 5, iteration: 2/100, moves: 0, cost: 43366.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 6, iteration: 1/100, moves: 1776, cost: 42595.0
Run 6, iteration: 2/100, moves: 1106, cost: 42361.0
Run 6, iterati

,data_index,cluster
0,28809,0
1,4690,0
2,32842,0
3,59695,0
6,42286,0
7,9209,0
8,39031,0
11,10746,0
15,18381,0
16,7378,0


In [59]:
df_indx=cluster_map[cluster_map.cluster == 1].index.values.astype(int)
df_indx
#final_clustering_df.iloc[df_indx,:]

array([    5,     9,    10, ..., 20203, 20238, 20240])

In [113]:
###### K_mean for Price ####### 
from sklearn.cluster import KMeans
K_pricemean=KMeans(n_clusters=5, n_init=10) 

#final_clustering_df['Price']
Price_Data= pd.DataFrame(final_clustering_df['Price'])
K_pricemean.fit(Price_Data)

# Get the cluster centroids
print(K_pricemean.cluster_centers_)

# Get the cluster labels
print(K_pricemean.labels_)
Price_Data.index.values

[[  23.64629682]
 [ 242.66509434]
 [  88.39564723]
 [ 578.00125   ]
 [   7.43746935]]
[0 4 0 ..., 0 0 4]


array([28809,  4690, 32842, ..., 46132, 42710, 30341], dtype=int64)

In [114]:
#Price clusters
cluster_Price = pd.DataFrame()
cluster_Price['data_index'] = Price_Data.index.values
cluster_Price['cluster'] = K_pricemean.labels_

#Print clusters
df_indx=cluster_Price[cluster_Price.cluster == 1].index.values.astype(int)
dd=Price_Data.iloc[df_indx,:]
list(dd['Price'])

def priceclust1(cluster_id):
    
    df_indx=cluster_Price[cluster_Price.cluster == cluster_id].index.values.astype(int)
    df=final_clustering_df.iloc[df_indx,:]
    clust1_price=list(df['Names'])
    
    return clust1_price

priceclust1(1)

['Furby Furblings Creature Special Feature Plush Toy (Rainbow)',
 'GIANT GODZILLA 2014 (24" tall)',
 'Lord of the Rings: Return of the King Chess Set',
 'Wilesco D20 Steam Engine',
 'God of War Kratos 7" Action Figure',
 'Halo 19206 "Micro Ops Boxed Series 1 High Ground Tower and Missile Launcher with Two Spartans Figures (Small)',
 "LEGO Star Wars 9516: Jabba's Palace",
 'Land Rover 1:32 Scale RMZ Defender (Black)',
 'LEGO Technic 42008: Service Truck',
 'Hello Kitty TY Beanie 6" Plush Union Jack Dress',
 'DC Comics 1:6 Scale Icons Batman Statue',
 'LEGO Power Miners 8188: Fire Blaster',
 'KidKraft 17826 Bucket Top Mountain Wooden Train Track Set for Kids (Classic Railway with Accessories Included), 61 Piece',
 'Fireman Sam Deluxe Fire Station Playset',
 'Square Enix Halo Reach Play Arts Kai Volume 1 Emile Figure',
 'Mega Bloks Toy - Halo Covenant Spectre Ambush Playset Including Figures',
 'LEGO REBEL BLOCKADE RUNNER 10019',
 'Neca Gremlins Series 1 Action Figure George',
 'Large Gre

In [93]:
# K_means for rating
from sklearn.cluster import KMeans
K_ratingmean=KMeans(n_clusters=5, n_init=10) 

#final_clustering_df['Price']
Rating_Data= pd.DataFrame(final_clustering_df['Rating'])
K_ratingmean.fit(Rating_Data)

# Get the cluster centroids
print(K_ratingmean.cluster_centers_)

# Get the cluster labels
print(K_ratingmean.labels_)
Rating_Data.index.values

[[ 4.26135647]
 [ 0.01065277]
 [ 2.88509969]
 [ 4.8118287 ]
 [ 3.64170013]]
[1 0 1 ..., 1 3 0]


array([28809,  4690, 32842, ..., 46132, 42710, 30341], dtype=int64)

In [106]:
#Rating clusters
cluster_rating = pd.DataFrame()
cluster_rating['data_index'] = Rating_Data.index.values
cluster_rating['cluster'] = K_ratingmean.labels_

###### Print clusters
df_indx=cluster_rating[cluster_rating.cluster == 1].index.values.astype(int)
dd=Rating_Data.iloc[df_indx,:]
list(dd['Rating'])

def ratingclust1(cluster_id):
    
    df_indx=cluster_rating[cluster_rating.cluster == cluster_id].index.values.astype(int)
    df=final_clustering_df.iloc[df_indx,:]
    clust1_rating=list(df['Names'])
    
    return clust1_rating

ratingclust1(4)

['LEGO Technic 42026: Black Champion Racer',
 'Sonic The Hedgehog 3-inch Free Riders Action Figure Jet',
 'Disney Princess Trefl 916 15299 Snow White Collage Puzzle',
 '5 Trashies in 5 Bins CDU',
 'Marvel Avengers Hero Mashers Hulk Action Figure',
 'LEGO Chima Crocodile Tribe Pack',
 'Ozbozz Pogo Stick Girls - New Colours',
 'Amscan 4.5 m Thomas Tank Foil Banner',
 'MADE4U Paint By Numbers Kits Canvas Mounted on Wood Frame with Brushes and Paints for Adults Children Seniors Junior DIY Beginner Level Acrylics Painting Kits on Canvas (Audrey Hepburn 8651)',
 'Physics Teaching Precision Optical Glass Prism',
 'Intex Swimming Pool Filter Cartridge Type H #29007',
 'HQ Quickstep 2 Stunt Kite (Black Rainbow)',
 'Thomas Take n Play James',
 'Alien 51382 7-Inch Series 4 Ripley White Nostromo Spacesuit Version Figure',
 'Furby Furblings Creature Special Feature Plush Toy (Rainbow)',
 'John Adams Sands Alive Animal Set',
 'Metallic Temporary Flash Tattoo Silver Black Gold Body Art Tattoos Neckla

In [112]:
len(final_clustering_df)
Popularity = [random.randint(0,30) for i in range(20242)]
len(Popularity)
final_clustering_df['Popularity'] = Popularity
final_clustering_df.head()

,Names,Price,Rating,Popularity
28809,cute pug dog print light switch cover sticker 143,21.741747,0.0,27
4690,"8 x 12"" Robin Reed Fill Your Own Christmas Cra...",12.480000,4.1,6
32842,Chinese Chess,21.741747,0.0,23
59695,Cribbage board - folding wood board 00180 by W...,10.390000,4.2,6
14895,LEGO Technic 42026: Black Champion Racer,59.990000,3.7,8


In [115]:
#K_mean for Popularity
from sklearn.cluster import KMeans
K_popularitymean=KMeans(n_clusters=5, n_init=10) 

#final_clustering_df['Price']
Popularity_Data= pd.DataFrame(final_clustering_df['Popularity'])
K_popularitymean.fit(Popularity_Data)

# Get the cluster centroids
print(K_popularitymean.cluster_centers_)

# Get the cluster labels
print(K_popularitymean.labels_)
Popularity_Data.index.values

[[  2.45794393]
 [ 21.51352731]
 [  8.48963399]
 [ 14.98505495]
 [ 27.48592631]]
[4 2 1 ..., 0 0 3]


array([28809,  4690, 32842, ..., 46132, 42710, 30341], dtype=int64)

In [132]:
#Popularity clusters
cluster_popularity = pd.DataFrame()
cluster_popularity['data_index'] = Popularity_Data.index.values
cluster_popularity['cluster'] = K_popularitymean.labels_

#Print clusters
df_indx=cluster_popularity[cluster_popularity.cluster == 1].index.values.astype(int)
dd=Popularity_Data.iloc[df_indx,:]
list(dd['Popularity'])

@anvil.server.callable
def popularityclust1(cluster_id):
    
    df_indx=cluster_popularity[cluster_popularity.cluster == cluster_id].index.values.astype(int)
    df=final_clustering_df.iloc[df_indx,:]
    clust1_popularity=list(df['Names'])
    
    return clust1_popularity

popularityclust1(0)

['Disney Princess Trefl 916 15299 Snow White Collage Puzzle',
 'Hama 3000 Beads Refill Tub, Glitter Mix (Midi Beads)',
 'Match Attax 2013/2014 Hugo Lloris Tottenham Star Player 13/14',
 'Tingacraft Acrylic Display Case (37.5 x 19 x 18.5 cm) for 1/35 Tank Model, Self-Assembly',
 'Marvel Avengers Hero Mashers Hulk Action Figure',
 'Dress Up America Child EMT Costume Set',
 'Ozbozz Pogo Stick Girls - New Colours',
 'Satin Alice Bands - 10mm - 3 Pack (Black)',
 "Orchard Toys Who's on the Farm? Jigsaw Puzzle",
 'Amscan 4.5 m Thomas Tank Foil Banner',
 'Ty Beanie Boos Bongo Orangutan 6" Plush',
 '17.75 Inch Walnut and Maple Chess Board',
 'Hama Large Frames Bead Kit',
 'Playmobil 5239 Duo Pack Vampires',
 'John Adams Sands Alive Animal Set',
 'Baker Ross Jungle Animal Latex Moulds for Children to Play and Display (Set of 5)',
 'Arshiner Kids Baby Wooden Doll House People Doll Toys for Girls Happy Doll Family of 7 People House Set',
 'Pokemon XY 2015 Summer Tin Hoenn Power Trading Cards',
 'I

In [128]:
import anvil.server
anvil.server.connect("JWQJJ3NSHPKG44TJYCOIRLIT-WPQX3T635PZE3EDS")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [2]:
############## DBSCAN Clustering ###################

import numpy as np
import pandas as pd
import math

#Program to to get clusters with DBSCAN clustering algorithms based on eps and minimum points.
#Input : Data list, Output: Clusters, -1 if observation/ point is noise in data.

def Get_DBSCAN(datalist, eps, minmum_Pts):                      # Function to get clusters from data with DBSCAN
    size = len(datalist)
    cluster_set = [0]*size                                      # Initialize a cluster point list with all 0s
    Cr = 0                                                      # Indicates a point hasn't been considered yet.
    
    

    ##### Check every observation if assigned to clusetr, if not then check eps and min points and accordingly add to the cluster
    for record in range(0, size):
        if not (cluster_set[record] == 0):                      #Initial skip as every value is 0
            continue
        
        neighbor_recordss = Lookforneighbors(datalist, record, eps)  
                                                                # get neighbors of given point 
        if len(neighbor_recordss) < minmum_Pts:                 # Check if neighbors are greater if not, mark point as noise
            cluster_set[record] = -1                        
        else: 
            Cr += 1                                             # Increment cluster number
            #expand_cluster(datalist, cluster_set, record, neighbor_recordss, Cr, eps, minmum_Pts)
            cluster_set[record] = Cr
            i = 0
            while i < len(neighbor_recordss):       
                newrecord = neighbor_recordss[i]
                if cluster_set[newrecord] == -1:                #Already marked -1 then assign it to cluster
                    cluster_set[newrecord] = Cr                     
                elif cluster_set[newrecord] == 0:               #If assigned initially 0 then assign it to cluster
                    cluster_set[newrecord] = Cr
                    newNeighborrecords = Lookforneighbors(datalist, newrecord, eps)
    
                    if len(newNeighborrecords) >= minmum_Pts:       #Again fetch the neighbors for neighbor of the current point.
                        neighbor_recordss = neighbor_recordss + newNeighborrecords
                i += 1              
    
    return cluster_set


def Lookforneighbors(datalist, record, eps):                   #Get neighbors based on given eps value         
    neighbor_recs = []
    size = len(datalist)
    for newrecord in range(0, size):                           #Check each observation in data and get observations within eps provided

        if np.linalg.norm(datalist[record] - datalist[newrecord]) < eps:
            neighbor_recs.append(newrecord)
            
    return neighbor_recs



In [3]:

final_clustering_df = pd.read_csv("Amazon_Association_final_clustering_df.csv")


In [28]:
# Price clustering with DBSCAN

#Price_Data= pd.DataFrame(final_clustering_df['Price'])
#Price_Data = Price_Data.head(1000)
#Price_Data.sort_values(by=['Price'])

Price_Data=pd.DataFrame(final_clustering_df.Price.unique())
Price_Data.rename(columns = {0:'Price'}, inplace = True) 


DBSCAN_Price=Get_DBSCAN(Price_Data.Price,0.5,10)

DBcluster_Price = pd.DataFrame()
DBcluster_Price['data_index'] = Price_Data.index.values
DBcluster_Price['cluster'] = DBSCAN_Price

In [50]:


'''df_indx=DBcluster_Price[DBcluster_Price.cluster == 5].index.values.astype(int)
dd=final_clustering_df.iloc[df_indx,:]
dd['Names']'''

def price_DBclust(cluster_id):
    
    df_indx=DBcluster_Price[DBcluster_Price.cluster == cluster_id].index.values.astype(int)
    df=final_clustering_df.iloc[df_indx,:]
    DBclust1_price=list(df['Names'])
    df_range=list(df['Price'])
    mn=min(df_range)
    mx=max(df_range)
    range_val=str(mn)+'-'+str(mx)
    
    return DBclust1_price, range_val

price_clusters= list()
range_values= list()
for i in range(1,6):
    getclust, range_val=price_DBclust(i)
    price_clusters.append(getclust[0:20])
    range_values.append(range_val)

price_clusters                       ######### Clusters product names list for all clusters
range_values


############# Updating the range list with proper values
print(range_values)

range_values[0] ='upto 770$'
range_values[1] ='Less than 25$'
range_values[2] = 'less than 50$'
range_values[3] = 'Less than 140$'
range_values[4] = 'Less than 35$'

print(range_values)

['0.01-770.0', '1.46-29.99', '2.76-48.43', '1.74-139.99', '1.42-33.14']
['upto 770$', 'Less than 25$', 'less than 50$', 'Less than 140$', 'Less than 35$']


In [51]:
#Rating clustering with DBSCAN

Rating_Data= pd.DataFrame(final_clustering_df['Rating'])
Rating_Data = Rating_Data.head(1000)
#Rating_Data=pd.DataFrame(final_clustering_df.Rating.unique())
#Rating_Data.rename(columns = {0:'Rating'}, inplace = True) 

DBSCAN_Rating=Get_DBSCAN(Rating_Data.Rating,0.5,10)

DBcluster_Rating = pd.DataFrame()
DBcluster_Rating['data_index'] = Rating_Data.index.values
DBcluster_Rating['cluster'] = DBSCAN_Rating

In [57]:
def rating_DBclust(cluster_id):
    
    df_indx=DBcluster_Rating[DBcluster_Rating.cluster == cluster_id].index.values.astype(int)
    df=final_clustering_df.iloc[df_indx,:]
    DBclust1_rating=list(df['Names'])
    df_range=list(df['Rating'])
    mn=min(df_range)
    mx=max(df_range)
    range_val=str(mn)+'-'+str(mx)
    
    return DBclust1_rating, range_val

rating_clusters= list()
rating_values= list()
for i in range(1,3):
    getclust, range_val=rating_DBclust(i)
    rating_clusters.append(getclust[0:20])
    rating_values.append(range_val)

rating_clusters         #Clusters product names list for all clusters
rating_values

print(rating_values)

# Updating the range list with proper values
rating_values[0] ='upto 2'
rating_values[1] ='3 to 5'


print(rating_values)

['1.8-5.0', '0.0-0.0']
['upto 2', '3 to 5']


In [59]:
# Popularity clustering with DBSCAN

Popularity = [random.randint(0,50) for i in range(20242)]
len(Popularity)
final_clustering_df['Popularity'] = Popularity
final_clustering_df.head()

# Popularity clustering with DBSCAN

Popularity_Data= pd.DataFrame(final_clustering_df['Popularity'])
Popularity_Data = Popularity_Data.head(2000)
DBSCAN_Popularity=Get_DBSCAN(Popularity_Data.Popularity,0.5,10)
DBcluster_Popularity = pd.DataFrame()
DBcluster_Popularity['data_index'] = Popularity_Data.index.values
DBcluster_Popularity['cluster'] = DBSCAN_Popularity

In [61]:
def popularity_DBclust(cluster_id):
    
    df_indx=DBcluster_Popularity[DBcluster_Popularity.cluster == cluster_id].index.values.astype(int)
    df=final_clustering_df.iloc[df_indx,:]
    DBclust1_popularity=list(df['Names'])
    df_range=list(df['Popularity'])
    mn=min(df_range)
    mx=max(df_range)
    range_val=str(mn)+'-'+str(mx)
    
    return DBclust1_popularity, range_val

popularity_clusters= list()
popularity_values= list()
for i in range(1,6):
    getclust, range_val=popularity_DBclust(i)
    popularity_clusters.append(getclust[0:20])
    popularity_values.append(range_val)

popularity_clusters                                  # Clusters product names list for all clusters
popularity_values

# Updating the range list with proper values

popularity_values[0] ='upto 35'
popularity_values[1] ='upto 25'
popularity_values[2] = 'upto 50'
popularity_values[3] = 'upto 20'
popularity_values[4] = 'upto 30'
popularity_clusters, popularity_values

([['Fridgemagic A3 Dry Wipe Magnetic Wipe Clean Board',
   'Match Attax EPL 15/16 Trading Card Starter Pack',
   'KATO 70148015 N Portram TLR 601 Portram, grün',
   'Chains and Bracelets ✮ 7 Designs ✮ DESIGNER METALLIC TEMPORARY FLASH TATTOO - Gold Silver Flash Tattoos',
   'Annual 2016 (My Little Pony)',
   'Unique Party 28481 - Glitz Black 40th Birthday Paper Napkins, Pack of 16',
   'Kinetic Sand Box Set',
   'Latitude 320622 Simon the Lion',
   'Wow Stuff 871219-12P Science Museum Giant Springy',
   'LEN Goodman\xa0–\xa0Celebrity Face Mask (Mask/Headpiece',
   'Fiesta Crafts Princess Tellatale Hand Glove Puppet',
   'GB eye LTD, The Who, Lyrics and Logos, Badge Pack, Aluminum, Multi-Colour',
   'Fireman Sam Complete Party Supplies Kit Blue For 16',
   'Stargazer Face and Body Glitter Stars Spectrum Party Festival Rave',
   'Oxford Diecast 76WO002 British Army Wolseley 18_85',
   'Paw Patrol Mini Award Medals',
   'S-39 EF510 Red Thunder (Tomica PlaRail Model Train) [Toy]',
   'Star